In [1]:
import os
import torch
import torch.nn.functional as F
from model import InvoiceGCN
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
# device agnostic code 

if torch.cuda.is_available():
    device = 'cuda'
else:
    device='cpu'
device

'cuda'

In [4]:
def load_train_test_split(save_fd):
    # import pdb; pdb.set_trace()
    train_data = torch.load(os.path.join(save_fd, 'train_data.dataset'))
    test_data = torch.load(os.path.join(save_fd, 'test_data.dataset'))
    return train_data, test_data


train_data, test_data = load_train_test_split(save_fd='sroie-2019/processed')

In [5]:
model = InvoiceGCN(input_dim=train_data.x.shape[1], chebnet=True).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.9)

train_data = train_data.to(device)
test_data = test_data.to(device)

_class_weights = compute_class_weight(class_weight='balanced',
                                      classes=train_data.y.unique().cpu().numpy(),
                                      y=train_data.y.cpu().numpy())

In [6]:
epochs = 2000
for epoch in range(0, epochs + 1):
    model.train()
    optimizer.zero_grad()

    loss = F.nll_loss(model(train_data), train_data.y - 1,
                      weight=torch.FloatTensor(_class_weights).to(device))
    loss.backward()
    optimizer.step()

    with torch.no_grad():
        if epoch % 10 == 0:
            model.eval()

            # forward model
            for index, name in enumerate(['train', 'test']):
                _data = eval(f'{name}_data')
                y_pred = model(_data).max(dim=1)[1]
                y_true = (_data.y - 1)
                acc = y_pred.eq(y_true).sum().item() / y_pred.shape[0]

                y_pred = y_pred.cpu().numpy()
                y_true = y_true.cpu().numpy()
                print(f'{name} accuracy: {acc}')

                if name == 'test':
                    print("test")
                    cm = confusion_matrix(y_true, y_pred)
                    class_acc = cm.diagonal() / cm.sum(axis=1)
                    print(classification_report(y_true, y_pred))

            loss_val = F.nll_loss(model(test_data), test_data.y - 1)
            print(f'Epoch: {epoch:3d}, train_loss: {loss:.4f}, val_loss: {loss_val:.4f}')
            print('>' * 50)

train accuracy: 0.32284892874255594
test accuracy: 0.35549930971007826
test
              precision    recall  f1-score   support

           0       0.02      0.65      0.04        49
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00        17
           3       0.02      0.05      0.03       168
           4       0.93      0.37      0.52      4112

    accuracy                           0.36      4346
   macro avg       0.19      0.21      0.12      4346
weighted avg       0.88      0.36      0.50      4346

Epoch:   0, train_loss: 11.5788, val_loss: 2.0841
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
train accuracy: 0.18700800876172222
test accuracy: 0.1859180855959503
test
              precision    recall  f1-score   support

           0       0.10      0.78      0.18        49
           1       0.00      0.00      0.00         0
           2       0.02      0.24      0.04        17
           3       0.05      0.83      0.0